## 문서 단어 행렬(Document-Term Matrix,DTM)

DTM : BoW 표현을 다수의 문서에 대해서 행렬로 표현

문서 1 : 000101100   
문서 2 : 000110100  
문서 3 : 011020000  

장점  
간단하게 표현 가능  
문서들을 서로 비교 가능함  

한계  
희소 표현(대부분의 벡터가 0이 아닌 값보다 0인 값이 더 많음)  
단순 단어 빈도 수 기반 접근 (유사성 판단 근거 부족)  

## TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency)

방법  
DTM을 만든 후, TF-IDF 가중치를 부여

쓰임  
문서 유사도 구하는 작업  
검색 시스템에서 검색 결과의 중요도를 정하는 작업  
문서 내에서 특정 단어의 중요도를 구하는 작업  

1. tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수  
2. df(t) : 특정 단어 t가 등장한 문서의 수
3. idf(d,t): dt(t)에 반비례하는 수

## 파이썬으로 TF-IDF 직접 구현

In [1]:
import pandas as pd #DF 사용
from math import log #IDF 계산

In [2]:
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [3]:
N = len(docs) # 총 문서의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [5]:
#TF 구하기
result = []
for i in range(N): # 각 문서에 대해서 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [6]:
#IDF 구하기
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [7]:
#TF-IDF 구하기
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### 사이킷런을 이용해서 DTM과 TF-IDF 구하기

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스 각 단어의 빈도 수
print(vector.vocabulary_) # 각 단어의 인덱스

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [12]:
#실제 라이브러리의 TF-IDF는 조금 더 조정된 다른 식을 사용한다.(값이 약간 다를 수 있다.)
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
